# Test 10 Posts First

In [ ]:
import time
import tweepy
import json

# Set your Twitter API credentials
API_KEY = 'your_api_key'
API_SECRET = 'your_api_secret'
BEARER_TOKEN = 'your_bearer_token'

# Initialize the client
client = tweepy.Client(bearer_token=BEARER_TOKEN)

# Search parameters
query = 'AI art ethical -is:retweet'  # Search keyword, excluding retweets
max_results = 10  # Number of tweets per request (maximum 100)
total_tweets_to_fetch = 10  # Total number of tweets to fetch

tweets = []  # To store fetched tweets
next_token = None  # Pagination token
pause_time = 3 * 60  # Pause time (3 minutes)

while len(tweets) < total_tweets_to_fetch:
    try:
        # Call the API to fetch tweets
        response = client.search_recent_tweets(
            query=query,
            max_results=max_results,
            next_token=next_token
        )

        # If data is returned, add it to the result
        if response.data:
            tweets.extend(response.data)
            print(f"Total tweets fetched so far: {len(tweets)}")
        else:
            print("No more tweets to fetch.")
            break

        # Get the next page token
        next_token = response.meta.get('next_token')
        if not next_token:
            print("All pages have been retrieved.")
            break

    except tweepy.TooManyRequests:
        # Handle rate limit errors (HTTP 429)
        print(f"Rate limit reached, pausing for {pause_time // 60} minutes...")
        time.sleep(pause_time)  # Pause for 3 minutes and then continue

    except Exception as e:
        # Handle other exceptions
        print(f"An error occurred: {e}")
        break

# Ensure no duplicate tweets in the result
tweets = list({tweet.id: tweet for tweet in tweets}.values())

# Convert the tweets to a dictionary list for JSON serialization
tweet_data = [{"id": tweet.id, "text": tweet.text, "created_at": str(tweet.created_at)} for tweet in tweets]

# Save tweets to a JSON file
output_file = 'tweets.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(tweet_data, f, ensure_ascii=False, indent=4)

print(f"A total of {len(tweets)} tweets were fetched and saved to {output_file}.")
